Import needed packages

In [ ]:
import scipy
from os import listdir
import os
from os.path import isfile, join
import PIL
from PIL import Image
import pydicom
import math 
from pathlib import Path
import copy
import operator
import imageio
import matplotlib.pyplot as plt
import pandas as pd
import csv
import cv2
import tensorflow as tf
import keras
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import regularizers
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam# use ADAM if it doesnt work
from tensorflow.keras.optimizers import legacy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import MaxPooling2D
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import visualkeras

Extract labels from excel file: ESCAPENA1 data for CIPAC 2022.xlsx

In [ ]:
df = pd.read_excel('05-Scripts/ESCAPENA1 data for CIPAC 2022.xlsx')
values = df.loc[df["Recanalized?"] == "Yes", "siteangside(1=L, 2=R)"].tolist()

vl=values[1:81]

labels = [x - 1.0 for x in vl]
labels = [int(x) for x in labels]
labels=np.array(labels)
#print(labels)

### Import data from folder IM1

In [ ]:
list_files = glob.glob("/Shared/CipacProcessing/Projects/RF_ML_ENA1/IM1/*.jpg")

#newL=[s.split('/')[-1] for s in list_files]
#print(newL)
new_file_list = [file_path for file_path in list_files if file_path.find('Oc') == -1]


pixel_data=[]
for name in new_file_list:
    try:
        img = Image.open(name).convert('L')  # Convert image to grayscale
    except FileNotFoundError:
        print(f"File not found: {name}")
        continue
    if img.size == (1024, 1024):
        # Apply histogram equalization
        img = cv2.equalizeHist(np.array(img))
        # Apply gamma correction
        img = np.array(255 * (img / 255) ** 0.2, dtype=np.uint8) # PLAY WITH THIS (0.85) - lower is lighter image
        pixel_array = np.array(img)
        #min-max normalization
        min_value = np.min(pixel_array)
        max_value = np.max(pixel_array)
        pixel_array = (pixel_array - min_value) / (max_value - min_value + 1e-12)        
        pixel_data.append(pixel_array.reshape(pixel_array.shape[0], pixel_array.shape[1], 1))

pixel_data=np.array(pixel_data)



Create train/val/test sets

In [ ]:
def split_patients(subjects, ratios):

    number_of_patients = len(subjects)
    train_size = int(number_of_patients*ratios[0])
    test_size = number_of_patients - train_size

    val_size = int(train_size*ratios[1])
    train_size -= val_size

    return train_size, val_size, test_size
ratios = [0.8, 0.2]#train/val
train_size, val_size, test_size = split_patients(pixel_data,ratios)
pixel_array_train = pixel_data[:train_size]
pixel_array_val = pixel_data[train_size:train_size+val_size]
pixel_array_test = pixel_data[train_size+val_size:]




labels_train = labels[:train_size]
labels_val = labels[train_size:train_size+val_size]
labels_test = labels[train_size+val_size:]
print(pixel_array_train.shape, labels_train.shape)
print(pixel_array_val.shape, labels_val.shape)
print(pixel_array_test.shape, labels_test.shape)

### Cell to print out images to see what they look like

In [ ]:
print(np.min(pixel_array_train[12]), np.max(pixel_array_train[12]))
print(np.min(pixel_array_train[13]), np.max(pixel_array_train[13]))
plt.subplot(1,2,1)
plt.title(labels_train[22])
plt.imshow(pixel_array_train[22], cmap="gray")
plt.axis("off")
plt.subplot(1,2,2)
plt.title(labels_train[23])
plt.imshow(pixel_array_train[23], cmap="gray")
plt.axis("off")
plt.show()

### Data Augmentation for Train set

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=5,# Rotate image by up to 10 degrees
    width_shift_range=0.01,  # Shift image horizontally by up to 5% of the image size
    height_shift_range=0.01,  # Shift image vertically by up to 5% of the image size
    zoom_range=0.01,  # Zoom in or out by up to 5%
    horizontal_flip=False,  
    vertical_flip=False,
    # fill_mode='nearest',  # Fill missing pixels with the nearest value
    # preprocessing_function=preprocess_input  # Preprocess input using ResNet50 preprocessing
)

# no augmentation for val set
datagen_val = ImageDataGenerator()

### Define the Model & visualize it

In [ ]:
input_shape = (1024, 1024, 1)

# model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
visualkeras.layered_view(model, scale_xy=0.3,legend=True, spacing=30 )

### Define callbacks

In [ ]:
# Define the callbacks
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.12, patience=7, min_lr=1e-8)

model_chck_point_path = "/Shared/CipacProcessing/Projects/RF_ML_ENA1/my_models/Final_models/mh1.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_chck_point_path, mode = 'auto', monitor='val_loss',verbose=0, save_best_only=True)


### Compile model

In [ ]:
# Compile
opt = legacy.Adam(lr=1e-4) 
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Train model and save best model checkpoint

In [ ]:
# Train 
batch_size_ = 52
hist = model.fit(datagen.flow(pixel_array_train, labels_train,batch_size=52,shuffle=True), 
                 epochs=20,verbose=1, batch_size=52, 
                 validation_data=datagen.flow(pixel_array_val, labels_val, batch_size=12,shuffle=True), 
                 callbacks=[checkpoint, reduce_lr])


### Print results

In [ ]:
model_load = keras.models.load_model(model_chck_point_path)
test_loss, test_acc = model_load.evaluate(pixel_array_test, labels_test, verbose=0)

y_pred = model_load.predict(pixel_array_test)
binary_predictions = np.round(y_pred)

f1 = f1_score(labels_test, binary_predictions)

accuracy = accuracy_score(labels_test, binary_predictions)

print("score using .evaluate():",test_acc)
print("accuracy score:",accuracy)
print("f1-score:",f1)
print("----------------------------------------------------------------")
report = classification_report(labels_test, binary_predictions)
print (report)
print("----------------------------------------------------------------")
matrix = confusion_matrix(labels_test, binary_predictions)
print (matrix)

### Print confusion matrix

In [ ]:
class_names = ['left', 'right']

cmd = ConfusionMatrixDisplay(matrix, display_labels=class_names)
cmd.plot()

### Plot loss function

In [ ]:
plt.figure(figsize=(4,5), dpi=150)
plt.grid('on')
plt.title('Loss Function')
plt.plot(hist.history['loss'], 'b', lw=2, alpha=0.7, label='Training')
plt.plot(hist.history['val_loss'], 'r', lw=2, alpha=0.7, label='Val')
plt.legend(loc="upper right")
plt.show()

### Plot detection tradeoff

In [ ]:
def fpr_fnr_threshold(y_true, y_scores):
    thresholds = np.sort(np.unique(y_scores))
    fpr = np.zeros_like(thresholds)
    fnr = np.zeros_like(thresholds)
    
    for i, thresh in enumerate(thresholds):
        y_pred = (y_scores >= thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        fpr[i] = fp / (fp + tn)
        fnr[i] = fn / (fn + tp)

    return fpr, fnr, thresholds


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

pixel_array_test = np.array(pixel_array_test)  
predictions = model.predict(pixel_array_test)

fpr, tpr, thresholds = roc_curve(labels_test, predictions)

interpolate_fpr = np.linspace(0, 1, num=10)
interpolate_tpr = np.interp(interpolate_fpr, fpr, tpr)

roc_auc = auc(interpolate_fpr, interpolate_tpr)


plt.figure()
plt.plot(interpolate_fpr, interpolate_tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

fpr, fnr, thresholds = fpr_fnr_threshold(labels_test, predictions)

interpolated_fpr = np.linspace(0, 1, num=10)
interpolated_fnr = np.interp(interpolated_fpr, fpr, fnr)

plt.figure()
plt.plot(interpolated_fpr, interpolated_fnr, color='darkorange', lw=2)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('False Negative Rate')
plt.title('Detection Error Tradeoff')
plt.grid()
plt.show()


### Cross Validation

In [ ]:
full_pixel_array = np.concatenate((pixel_array_train, pixel_array_val, pixel_array_test), axis=0)
full_label_array = np.concatenate((labels_train, labels_val, labels_test), axis=0)
print(full_pixel_array.shape,full_label_array.shape)

In [ ]:
num_splits = 5
pixel_array_splits = np.array_split(full_pixel_array, num_splits)
label_array_splits = np.array_split(full_label_array, num_splits)

In [ ]:
datagen = ImageDataGenerator(
    #rotation_range=10,  # Rotate image by up to 10 degrees
    width_shift_range=0.05,  # Shift image horizontally by up to 5% of the image size
    height_shift_range=0.1,  # Shift image vertically by up to 5% of the image size
    zoom_range=0.1,  # Zoom in or out by up to 5%
    horizontal_flip=False,  # Flip image horizontally
    # vertical_flip=True,
    # fill_mode='nearest',  # Fill missing pixels with the nearest value
    # preprocessing_function=preprocess_input  # Preprocess input using ResNet50 preprocessing
)

datagen_val = ImageDataGenerator()

In [ ]:
num_splits = 5
scores_acc_eval = []
scores_acc_acc = []
scores_f1 = []
classification_reports = []
confusion_matrices = []

# Change folder name as needed

model_folder = "/Shared/CipacProcessing/Projects/RF_ML_ENA1/my_models/Final_models/"
model_basename = "mh_cross"

for val_index in range(num_splits):
    # select one split as validation set
    val_pixel_array = pixel_array_splits[val_index]
    val_label_array = label_array_splits[val_index]

    # combine remaining splits into training set
    train_pixel_array = np.concatenate(
        [split for i, split in enumerate(pixel_array_splits) if i != val_index],
        axis=0
    )
    train_label_array = np.concatenate(
        [split for i, split in enumerate(label_array_splits) if i != val_index],
        axis=0
    )

    # Shuffle training set and corresponding indices
    train_shuffle_indices = np.random.permutation(len(train_pixel_array))
    train_pixel_array = train_pixel_array[train_shuffle_indices]
    train_label_array = train_label_array[train_shuffle_indices]

    # Shuffle validation set and corresponding indices
    val_shuffle_indices = np.random.permutation(len(val_pixel_array))
    val_pixel_array = val_pixel_array[val_shuffle_indices]
    val_label_array = val_label_array[val_shuffle_indices]
    # train and evaluate model on this split

    # Define the data generators for this fold
    train_datagen = datagen.flow(train_pixel_array, train_label_array, batch_size=32, shuffle=True)
    val_datagen = datagen_val.flow(val_pixel_array, val_label_array, batch_size=32)
    
    opt = legacy.Adam(lr=1e-4)

    # Define the callbacks
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=6, min_lr=1e-8)
    # model_chck_point_path = "/Shared/CipacProcessing/Projects/RF_ML_ENA1/my_models/mymodelfin5.h5"
    model_chck_point_path = f"{model_folder}{model_basename}_split{val_index+1}.h5"
    print(model_chck_point_path)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_chck_point_path, mode = 'auto', monitor='val_loss',verbose=0, save_best_only=True)

    # Define and compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
    #load model first

    # Train the model for this fold
    hist = model.fit(train_datagen, 
                    steps_per_epoch=len(train_datagen),
                    epochs=20, 
                    verbose=1, 
                    validation_data=val_datagen,
                    callbacks=[checkpoint, reduce_lr])


    #load model first
    model = keras.models.load_model(model_chck_point_path)


    # Evaluate the model on the test set for this fold and store the accuracy score
    test_loss, test_acc = model.evaluate(val_pixel_array, val_label_array, verbose=0)
    scores_acc_eval.append(test_acc)

    y_pred = model.predict(val_pixel_array)
    binary_predictions = np.round(y_pred)

    f1 = f1_score(val_label_array, binary_predictions)
    scores_f1.append(f1)

    accuracy = accuracy_score(val_label_array, binary_predictions)
    scores_acc_acc.append(accuracy)

    report = classification_report(val_label_array, binary_predictions)
    classification_reports.append(report)

    matrix = confusion_matrix(val_label_array, binary_predictions)
    confusion_matrices.append(matrix)

In [ ]:
print("scores acc eval:",scores_acc_eval)
print("accuracy scores:",scores_acc_acc)
print("f1-scores:",scores_f1)

In [ ]:
print("------------------------ RUN 1 (fold 1) ------------------------")
print(classification_reports[0])
print("------------------------ RUN 2 (fold 2) ------------------------")
print(classification_reports[1])
print("------------------------ RUN 3 (fold 3) ------------------------")
print(classification_reports[2])
print("------------------------ RUN 4 (fold 4) ------------------------")
print(classification_reports[3])
print("------------------------ RUN 5 (fold 5) ------------------------")
print(classification_reports[4])

# classification_reports
# confusion_matrices

In [ ]:
class_names = ['occluded', 'recanalized']

print("------------------------ RUN 1 (fold 1) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[0], display_labels=class_names)
cmd.plot()
print("------------------------ RUN 2 (fold 2) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[1], display_labels=class_names)
cmd.plot()
print("------------------------ RUN 3 (fold 3) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[2], display_labels=class_names)
cmd.plot()
print("------------------------ RUN 4 (fold 4) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[3], display_labels=class_names)
cmd.plot()
print("------------------------ RUN 5 (fold 5) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[4], display_labels=class_names)
cmd.plot()


In [ ]:
plt.figure(figsize=(4,5), dpi=150)
plt.grid('on')
plt.title('Loss Function')
plt.plot(hist.history['loss'], 'b', lw=2, alpha=0.7, label='Training')
plt.plot(hist.history['val_loss'], 'r', lw=2, alpha=0.7, label='Val')
plt.legend(loc="upper right")

plt.show()